<a href="https://colab.research.google.com/github/jylee2930/DB_Basic/blob/main/Pandas_Data_Analysis_%EC%9D%B8%EC%82%AC%EC%9D%B4%ED%8A%B8_%EB%B0%9C%EA%B2%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 탐색적 데이터 분석

In [ ]:
import pandas as pd

file_path = "/content/chipotle.tsv"
chipo = pd.read_csv(file_path, sep='\t')

print(chipo.shape)
print("---------------------------")
print(chipo.info())

# 1. head()

In [ ]:
chipo.head(10)

##2. 변수타입 변경 함수 : 변수명.astype(변수타입)

In [ ]:
##수치형 order_id를 범주형으로 변환
chipo['order_id'] = chipo['order_id'].astype(str)
chipo.describe()

In [ ]:
chipo['order_id'].unique() # order_id 를 모두 출력합니다.

In [ ]:
chipo['item_name'].unique() # item_name를 모두 출력합니다.

In [ ]:
print(len(chipo['order_id'].unique())) # order_id의 개수를 출력합니다.
print(len(chipo['item_name'].unique())) # item_name의 개수를 출력합니다.

In [ ]:
# 아이템 종류와 갯수 출력
item_count = chipo['item_name'].value_counts()
print(item_count)

In [ ]:
# 가장 많이 주문한 item_name : top 10을 출력합니다.
item_count = chipo['item_name'].value_counts()[:10]
print(item_count)


# 3. enumerate(리스트, start = 0)
  순서가 있는 자료형을 입력받았을 때, 인덱스와 값을 포함하여 리턴하는 함수   
  for 와 함께 사용


In [ ]:
for idx, (val, cnt) in enumerate(item_count.items()):
    print("Top", idx, ":", val, cnt)

In [ ]:
for idx, (val, cnt) in enumerate(item_count.items(), 1):
    print("Top", idx, ":", val, cnt)

# numpy의 tolist()는 array를 list로 바꿔줍니다.

In [ ]:
# item당 주문 개수를 출력합니다.
order_count = chipo.groupby('item_name')['quantity'].count().sort_values(ascending = False)
print(order_count) # item당 주문 개수를 출력합니다.

In [ ]:
print(len(order_count))

In [ ]:
# item당 주문 총량을 출력합니다.
item_quantity = chipo.groupby('item_name')['quantity'].sum()
print(item_quantity) # item당 주문 총량을 출력합니다.

In [ ]:
len(item_quantity)

In [ ]:
# 가장 많이 주문한 item_name만 출력합니다.
chipo['item_name'].value_counts().index.tolist()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

item_name_list = item_quantity.index.tolist() ##x축 범위 계산
x_pos = np.arange(len(item_name_list))
order_cnt = item_quantity.values.tolist() ##y축 범위 계산

plt.figure(figsize = (20,6) )
plt.bar(x_pos, order_cnt, align='center')
##plt.xticks(x_pos, item_name_list, rotation = 90) ##x축 아이템이름 설정
plt.xlabel('Item_list')
plt.ylabel('ordered_item_count')
plt.title('Distribution of all orderd item')
plt.show()

# Step3. 데이터 전처리 : 전처리 함수 사용하기

In [ ]:
# column 단위 데이터에 apply 함수로 특정 열만 전처리 적용합니다.
##item_price(범주형 자료)를 수치적 자료로 변환
## float(x[1:]) : $2.39 의 경우 $를 제외하고 2.39를 수치로 변환을 의미함.
chipo['item_price'] = chipo['item_price'].apply(lambda x: float(x[1:]))
chipo.describe()

# Step4. 탐색적 분석

In [ ]:
## 1. 주문 아이디별 주문 금액 기초 통계량 출력
chipo.groupby('order_id')['item_price'].sum().describe()

In [ ]:
# 1. 주문 아이디별 평균 계산금액을 출력
chipo.groupby('order_id')['item_price'].sum().mean()

In [ ]:
# 2. 한 주문에 10달러 이상 사용한 id를 출력합니다.
## 주문 아이디별로 가격의총합을 구한다.
chipo_orderid_group = chipo.groupby('order_id')['item_price'].sum()
print(chipo_orderid_group)


In [ ]:
## 총합이 10달러 이상인 값들만 results에 저장
results = chipo_orderid_group[chipo_orderid_group.item_price >= 10]

print(results)
print(results.index.values)

In [ ]:
# 3. 각 아이템의 가격을 계산합니다.
##1) 아이템을 1개만 주문한 주문 찾기
chipo_one_item = chipo[chipo.quantity == 1]
print(chipo_one_item)
##외국의 경우 같은 steak Burito 라고 해도 값이 다르기 때문에 평균이 필요하다.
##2)item_name 을 기준으로 가장 작은 값을 기본 가격으로 책정
price_per_item = chipo_one_item.groupby('item_name').min()
print(price_per_item)
price_per_item.sort_values(by = "item_price", ascending = False)[:10]

In [ ]:
## 아이템별 가격 분포도 그려보기
#1) 아이템을 리스트로 저장
item_name_list = price_per_item.index.tolist()
x_pos = np.arange(len(item_name_list))
y_pos = price_per_item['item_price'].tolist()

plt.figure(figsize = (20,6))
plt.bar(x_pos,y_pos)
plt.xticks(x_pos,item_name_list, rotation = 90)
plt.xlabel("Item_list")
plt.ylabel("Price")
plt.title("Distribution of All Ordered Items")
plt.legend()
plt.show()

In [ ]:
##4. 가장 비싼 주문에서 아이템이 총 몇개 팔렸는지 구하기
chipo.groupby('order_id').sum().sort_values(by = 'item_price', ascending=False)[:10]


In [ ]:
##5.Veggie Salad Bowl이 몇번 주문되었는지 구하기
chipo_salad = chipo[chipo.item_name == "Veggie Salad Bowl"]
print(chipo_salad.head(5) )
